In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Load Saumya's Model
# ✅ Load the saved TensorFlow 2 model
saumyamodel_path = Path (r"E:\ArtifactRemovalProject\Saumya NNArtifact\saved_model\NNArtifact_tf2")
saumyamodel = tf.saved_model.load(saumyamodel_path)

si = np.load(r"E:\ArtifactRemovalProject\data\modelinference\DOSEESC_EM09\11.20.2018\si.npy")

num_classes = 2
num_slices = si.shape[0] #32
x_size = si.shape[1] #64
y_size = si.shape[2] #64
spectra_size = si.shape[3] #512

BATCH_SIZE = x_size * y_size

ypred_array = np.zeros((num_slices, x_size, y_size, num_classes)) # 32,64,64,2

# ✅ Run inference using the loaded model
for z_slice in range(num_slices):
    print(z_slice)
    xtest = np.real(si[z_slice])
    xtest = xtest.reshape(x_size*y_size, spectra_size) #4096x512

    ypred_array[z_slice] = np.asarray(tf.nn.softmax(saumyamodel.predict(xtest))).reshape(x_size, y_size, num_classes)

# ── 10) Cleanup ────────────────────────────────────────────────────────────
del saumyamodel
tf.keras.backend.clear_session()

# Load My Model
# ── 1) Load your config to get model settings ───────────────────────────────
base_dir    = Path.cwd().parent
ensemble_cfg_path = base_dir / "results" / "ensembles" / "config.json"

with open(ensemble_cfg_path, "r") as f:
    cfg = json.load(f)

# Choose which ensemble to apply (e.g. 'fits_combined', 'fits+water', 'all_four')
exp_key = "all_four"  # or any key present in your config
info    = cfg[exp_key]

# ── 2) Load your saved ensemble model ───────────────────────────────────────
model = tf.keras.models.load_model(info["model_path"])
print(f"{exp_key} Model Loaded")

# ── 3) Load the spatial-spectral data ───────────────────────────────────────
water = np.load(r"E:\ArtifactRemovalProject\data\modelinference\DOSEESC_EM09\11.20.2018\siref.npy")
fit1 = np.load(r"E:\ArtifactRemovalProject\data\modelinference\DOSEESC_EM09\11.20.2018\midasfit.npy")
fit2    = np.load(r"E:\ArtifactRemovalProject\data\modelinference\DOSEESC_EM09\11.20.2018\nnfit.npy")

num_slices, H, W, S = fit1.shape  # (32,64,64,512)

# ── 4) Flatten spatial dims for vectorized preprocessing ───────────────────
raw = si.reshape(-1, S)
water = water.reshape(-1, S) # shape (N,512)
fit1 = fit1.reshape(-1, S) # shape (N,512)
fit2   = fit2.reshape(-1, S)     # shape (N,512)

# 4a) z-score normalization function
def zscore_per_spectrum(x, eps=1e-6):
    mu  = x.mean(axis=1, keepdims=True)
    std = x.std(axis=1, keepdims=True) + eps
    return (x - mu) / std

# 4b) normalize each channel
raw = zscore_per_spectrum(raw)
fit1   = zscore_per_spectrum(fit1)
fit2   = zscore_per_spectrum(fit2)

# uncomment for water
wat_log = np.log10(np.abs(water) + 1e-6)
wmin    = wat_log.min(axis=1, keepdims=True)
wmax    = wat_log.max(axis=1, keepdims=True) + 1e-6
water = (wat_log - wmin) / (wmax - wmin)

# ── 5) Build model input tensor ────────────────────────────────────────────
# Channels order from config, e.g. ["raw","water"]
channels = info["channels"]
# Stack in the correct order
channel_arrays = {
    "raw": raw,
    "water": water,
    "fit1": fit1,
    "fit2": fit2
    # add "fit1 - midas" and "fit2 - nnfit" if needed and loaded
}
# concatenate into shape (N, 512, n_ch)
X = np.stack([channel_arrays[ch] for ch in channels], axis=-1).astype("float32")

# ── 6) Run inference ────────────────────────────────────────────────────────
probs = model.predict(X, batch_size=4096).ravel()

# ──  Cleanup ────────────────────────────────────────────────────────────
del model
tf.keras.backend.clear_session()

# ── 7) Threshold predictions ───────────────────────────────────────────────
thr   = info["threshold"]
preds = (probs >= thr).astype(int)

# ── 8) Reshape back to spatial volume ───────────────────────────────────────
probs_vol = probs.reshape(num_slices, H, W)
preds_vol = preds.reshape(num_slices, H, W)

preds_vol = preds_vol ^ 1
